# Create atom data
---
This notebook produces multiple atom data files for TARDIS on a single run.

## Notes
- Remember to download and extract the Chianti Database:

```
cd $HOME/Downloads
wget http://www.chiantidatabase.org/download/CHIANTI_9.0.1_data.tar.gz
mkdir chianti
tar -xvzf CHIANTI_9.0.1_data.tar.gz -C chianti
```

In [ ]:
import os

os.environ['XUVTOP'] = os.path.expanduser('~/Downloads/chianti')

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData
from multiprocessing import Pool

In [ ]:
OUTPUT_DIR = os.path.expanduser('~/Downloads/tardis-data')
GFALL_IONS = 'H-Zn'
CHIANTI_IONS = (None, 'Si 1', 'Si 2', 'S 0', 'S 1', 'S 2', 'Ca 1', 'Fe 1', 'Fe 2', 'Ni 1', 'Si 1-2; S 0-2; Ca 1; Fe 1-2; Ni 1')

In [ ]:
def create_atom_data(gfall_ions, chianti_ions, output_dir):
  
    parsers = []
    parsers.append(NISTWeightsComp())
    parsers.append(NISTIonizationEnergies(gfall_ions))
    parsers.append(GFALLReader(gfall_ions))
    parsers.append(KnoxLongZeta())

    if chianti_ions:
        parsers.append(ChiantiReader(chianti_ions, priority=20))
        ions_string = chianti_ions.replace(' ', '_').replace(';', '')
        output_file = 'kurucz_cd23_latest_chianti_{}.h5'.format(ions_string)

    else:
        output_file = 'kurucz_cd23_latest.h5'

    os.makedirs(output_dir, exist_ok=True)
    fname = os.path.join(output_dir, output_file)
    
    atom_data = TARDISAtomData(*parsers)
    atom_data.to_hdf(fname)

    return

In [ ]:
pool = Pool(6)

In [ ]:
_ = [ pool.apply_async(create_atom_data, args=(GFALL_IONS, ions, OUTPUT_DIR)) for ions in CHIANTI_IONS ]